In [1]:
import scipy.io
data = scipy.io.loadmat('data/url.mat')

**create list for labels and data, where one entry is the data for the day with this index**

In [2]:
import numpy as np

num_of_days = 120
X, Y = [], []

for i in range(num_of_days):
    day_data = data["Day" + str(i)]
    X.append(day_data[0][0][0])
    Y.append(day_data[0][0][1])

**flatten y to not be a nested array**

In [3]:
# y-data are lists of numpy-arrays

for i in range(len(Y)):
    Y[i] = [element for sublist in Y[i] for element in sublist]
   
# y-data becomes a list of lists

### Train on data of previous day and predict of successive day

In [6]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

for i in range(num_of_days - 1):
    # i being the current day.
    clf = LinearSVC()
    clf.fit(X[i], Y[i])
    
    # i + 1 being the next day on which the model is being tested on. 
    Y_preds = clf.predict(X[i + 1])
    print("Error-rate Day {}   : {}".format(i + 1,1 - accuracy_score(Y[i + 1], Y_preds)))

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Error-rate Day 1   : 0.02554999999999996


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Error-rate Day 2   : 0.02024999999999999


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Error-rate Day 117   : 0.023900000000000032


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Error-rate Day 118   : 0.02429999999999999
Error-rate Day 119   : 0.022599999999999953


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
